In [ ]:
import numpy as np
import pandas as pd
import os

from glob import glob

import matplotlib.pyplot as plt

from tensorflow.keras.layers import BatchNormalization,Conv2D,Dense,Dropout,MaxPool2D,Flatten,AveragePooling2D,GlobalMaxPool2D
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [ ]:
IMAGE_SIZE=[256,256]

In [ ]:
train_path = '/content/data/train'
test_path = '/content/data/train'

In [ ]:
train_files = glob(train_path + "/*/*.jpg")
test_files = glob(test_path + "/*/*.jpg")

In [ ]:
plt.imshow(image.load_img(np.random.choice(train_files)))

In [ ]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.1,
    zoom_range = 0.2,
    vertical_flip=True)

In [ ]:
train_gen = gen.flow_from_directory('data/train',target_size=IMAGE_SIZE,shuffle=True)
test_gen = gen.flow_from_directory('data/test',target_size=IMAGE_SIZE,shuffle=True)

In [ ]:
train_gen.class_indices

In [ ]:
from tensorflow.keras.applications import VGG16 as Pretrained_Model
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:

ptm = Pretrained_Model(
    input_shape=IMAGE_SIZE + [3],
    weights='imagenet',
    include_top =False)

In [ ]:
ptm.trainable = False

In [ ]:
x = Flatten()(ptm.output)
prediction = Dense(3,activation='softmax')(x)

In [ ]:
model = tf.keras.models.Model(inputs=ptm.input,outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
Early_Stopping = tf.keras.callbacks.EarlyStopping(verbose=3,patience=3)
model.fit(train_gen,epochs=12,
          validation_data=test_gen)

In [ ]:
losses = pd.DataFrame(model.history.history)
losses

In [ ]:
plt.plot(losses['accuracy'],color='red')
plt.plot(losses['val_accuracy'])
plt.legend()

In [ ]:
plt.plot(losses['loss'],color='red')
plt.plot(losses['val_loss'])
plt.legend()

In [ ]:
name_labels_df = pd.DataFrame(list(test_gen.class_indices.items()),columns = ['names','labels'],index=None)

In [ ]:
name_labels_df

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image


def predict_random_image():
  img = image.load_img(np.random.choice(test_files), target_size=IMAGE_SIZE)
  x = image.img_to_array(img)
  plt.imshow(img)
  x = np.expand_dims(x, axis=0) /255.0
  result_df = pd.DataFrame(list(model.predict(x)[0]),index=name_labels_df.names.values,columns=['result'])
  print(result_df.result.idxmax())

In [ ]:
predict_random_image()

In [ ]:
predict_random_image()

In [ ]:
predict_random_image()

In [ ]:
model.save("image_classifier_model.h5")

In [ ]:
model.save('image_classifier_model')